In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3363/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_9_9_24,0.977380,0.914527,0.985196,0.968657,0.025033,0.049302,0.030550,0.040477,"Hidden Size=[60, 30], regularizer=0.02, learni..."
1,model_9_9_23,0.977140,0.916317,0.985508,0.969315,0.025299,0.048269,0.029905,0.039627,"Hidden Size=[60, 30], regularizer=0.02, learni..."
2,model_9_9_22,0.976881,0.918112,0.985821,0.969974,0.025586,0.047234,0.029259,0.038776,"Hidden Size=[60, 30], regularizer=0.02, learni..."
3,model_9_9_21,0.976600,0.919910,0.986132,0.970633,0.025897,0.046197,0.028617,0.037925,"Hidden Size=[60, 30], regularizer=0.02, learni..."
4,model_9_9_20,0.976298,0.921702,0.986442,0.971290,0.026231,0.045163,0.027979,0.037076,"Hidden Size=[60, 30], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...
590,model_11_8_1,0.893181,0.777404,0.943202,0.882143,0.118217,0.318156,0.020772,0.178211,"Hidden Size=[60, 30], regularizer=0.1, learnin..."
591,model_5_2_18,0.893135,0.658437,0.922089,0.877300,0.118268,0.158628,0.141503,0.150569,"Hidden Size=[45, 30], regularizer=0.02, learni..."
600,model_3_7_0,0.892020,0.971991,0.978180,0.977064,0.119502,0.022456,0.032443,0.027155,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
611,model_5_2_17,0.890597,0.658175,0.922810,0.877750,0.121076,0.158750,0.140194,0.150017,"Hidden Size=[45, 30], regularizer=0.02, learni..."
